# Case study

In the case study we demonstrate how to generate counterfactual explanations by using our library on Statlog (German Credit Data) Data Set from UCI ML repository.  

In [ ]:
from tensorflow import keras
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
np.random.seed(44)

### Data and model loading

Statlog (German Credit Data) was gathered from UCI ML repository and consists of 20 features and 61 columns (most of them are represented by one-hot encoding). The dataset is wrapped in GermanData class, for easier use. 

Additionally, we load a pretrained keras model (simple logistic regression with 2 outputs) which will be used for prediction making.

In [ ]:
from data import GermanData

german_data = GermanData('data/datasets/input_german.csv', 'data/datasets/labels_german.csv')
model = keras.models.load_model('models/model_german')

In [3]:
german_data.input.sample(5)

,duration,credit,installment_percent,residence_duration,age,existing_credits,people_maintained,account_status_0..200 DM,account_status_< 0 DM,account_status_>= 200 DM,...,job_unskilled - resident,phone_none,"phone_yes, registered under the customers name",foreign_no,foreign_yes,employment_1..4 years,employment_4..7 years,employment_< 1 year,employment_>= 7 years,employment_unemployed
99.0,20.0,7057.0,3.0,4.0,36.0,2.0,2.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
488.0,10.0,1418.0,3.0,2.0,35.0,1.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
206.0,12.0,1935.0,4.0,4.0,43.0,3.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
939.0,24.0,6842.0,2.0,4.0,46.0,2.0,2.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
729.0,24.0,1275.0,2.0,4.0,36.0,2.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


### Test sample selection

We select one instance, for which we want to calculate the counterfactual. 

In [4]:
X_test = german_data.input.iloc[0]
X_test

duration                    6.0
credit                   1169.0
installment_percent         4.0
residence_duration          4.0
age                        67.0
                          ...  
employment_1..4 years       0.0
employment_4..7 years       0.0
employment_< 1 year         0.0
employment_>= 7 years       1.0
employment_unemployed       0.0
Name: 0.0, Length: 61, dtype: float64

### Data scaling

For the model to work, the dataset has to be standardized. The GermanData scale function uses StandardScaler from sklearn.

In [5]:
X_test_scaled = german_data.scale(X_test)
X_test_scaled

duration                -1.242314
credit                  -0.753586
installment_percent      0.936622
residence_duration       1.024474
age                      2.796337
                           ...   
employment_1..4 years   -0.707770
employment_4..7 years   -0.456573
employment_< 1 year     -0.454573
employment_>= 7 years    1.676163
employment_unemployed   -0.252646
Length: 61, dtype: float64

Now, we evaluate the model on the test sample.

In [6]:
model.predict(np.expand_dims(X_test_scaled, axis=0))

array([[0.00789263, 0.99210733]], dtype=float32)

These outputs  are interpreted as the model prediction of the testing instance to class 1. For this credit data it means a bad (not paying loans) class of customers.

### Counterfactual explanations generation

Here we demonstrate how to generate counterfactual explanations using CADEX, FIMAP and ECE methods implemented in our library.

#### FIMAP


In [7]:
from counterfactuals.explainers import Fimap

model_predictions = model.predict(german_data.X_train)
model_predictions = np.argmax(model_predictions, axis=1)
fimap = Fimap()
fimap.fit(german_data.X_train, model_predictions)

cf_fimap = fimap.generate(X_test)


Training s

Training g


In [8]:
model.predict(german_data.scale(cf_fimap))

array([[0.6010637, 0.3989363]], dtype=float32)

The class predicted for the counterfactual is 0, meaning good credit score.

In [9]:
from counterfactuals.visualization import show

show(X_test, cf_fimap)

,X,X',change
duration,6.0,8.034,2.034
credit,1169.0,1168.541,-0.459
installment_percent,4.0,3.590,-0.410
residence_duration,4.0,5.268,1.268
age,67.0,65.640,-1.360
...,...,...,...
employment_1..4 years,0.0,1.168,1.168
employment_4..7 years,0.0,1.718,1.718
employment_< 1 year,0.0,-0.036,-0.036
employment_>= 7 years,1.0,1.227,0.227


We can see two things wrong with the generated counterfactual: 
- the categorical variables, originally represented in one-hot encoding, were changed to values different than 0 or 1
- the value of age variable decreased, resulting in poor quality of the counterfactual - we can't recommend someone to decrease their age in order to obtain credit

To fix it we can use constraints, which can be defined either in code or in spreadsheets (which can be used by users not familiar with programming).

In [10]:
from counterfactuals.constraints import OneHot, ValueMonotonicity

constraints = [OneHot("account_status", 7, 10), 
               OneHot("credit_history", 11, 15),
               OneHot("purpose", 16, 25), 
               OneHot("savings", 26, 30), 
               OneHot("sex_status", 31, 34),
               OneHot("debtors", 35, 37), 
               OneHot("property", 38, 41),
               OneHot("other_installment_plans", 42, 44), 
               OneHot("housing", 45, 47), 
               OneHot("job", 48, 51),
               OneHot("phone", 52, 53), 
               OneHot("foreign", 54, 55), 
               OneHot("employment", 56, 60),
               ValueMonotonicity(['age'], "increasing")
              ]

In [11]:
fimap = Fimap(constraints=constraints)
fimap.fit(german_data.X_train, model_predictions)

cf_fimap_constraints = fimap.generate(X_test)


Training s

Training g


In [12]:
model.predict(german_data.scale(cf_fimap_constraints))

array([[0.01989712, 0.98010284]], dtype=float32)

As FIMAP is a method that doesn't guarantee finding a true counterfactual (with a change in prediction), we should always check whether the prediction changed. Here, we'd have to tune the hyperparameters to find a true counterfactual.

In [13]:
show(X_test, cf_fimap_constraints, constraints=constraints)

,X,X',change,constraint
duration,6.0,6.319,0.319,
credit,1169.0,1169.343,0.343,
installment_percent,4.0,3.511,-0.489,
residence_duration,4.0,3.665,-0.335,
existing_credits,2.0,2.18,0.18,
people_maintained,1.0,0.251,-0.749,
account_status,account_status_< 0 DM,account_status_no checking account,account_status_no checking account,OneHot
purpose,purpose_radio/television,purpose_car (new),purpose_car (new),OneHot
savings,savings_unknown/ no savings account,savings_< 100 DM,savings_< 100 DM,OneHot
sex_status,sex_status_male: single,sex_status_male: married/widowed,sex_status_male: married/widowed,OneHot


The constraints column shows what constraints have been placed on the given attribute. If these constraints were not met, it would be marked with an asterisk.

#### CADEX

For CADEX, we can either pass scaled instance and then unscale the obtained counterfactual or pass transform and inverse_transform parameters to the constructor.

In [14]:
from counterfactuals.explainers import Cadex 

cadex = Cadex(model, transform=german_data.scale, inverse_transform=german_data.unscale)
cf = cadex.generate(X_test)

In [15]:
model.predict(german_data.scale(cf))

array([[0.50213647, 0.49786362]], dtype=float32)

In [16]:
show(X_test, cf)

,X,X',change
duration,6.0,30.311,24.311
account_status_< 0 DM,1.0,1.922,0.922
account_status_no checking account,0.0,-0.998,-0.998
debtors_guarantor,0.0,-0.459,-0.459
foreign_yes,1.0,1.397,0.397


For CADEX we can also use constraints:

In [17]:
cadex = Cadex(model, n_changed=10, transform=german_data.scale, inverse_transform=german_data.unscale, constraints=constraints)
cf = cadex.generate(X_test)

In [18]:
model.predict(german_data.scale(cf))

array([[0.59131646, 0.4086835 ]], dtype=float32)

In [19]:
show(X_test, cf, constraints=constraints)

,X,X',change,constraint
duration,6.0,21.716,15.716,
credit,1169.0,4795.549,3626.549,
purpose,purpose_radio/television,purpose_car (new),purpose_car (new),OneHot
savings,savings_unknown/ no savings account,savings_< 100 DM,savings_< 100 DM,OneHot
other_installment_plans,other_installment_plans_none,other_installment_plans_bank,other_installment_plans_bank,OneHot
housing,housing_own,housing_rent,housing_rent,OneHot


### ECE 
We can use ECE to select the best counterfactuals - we'll run it using 10 explainers, 5 FIMAPs and 5 CADEXs with different parameter values. 

In [ ]:
fimaps = []
fimap_hyperparameters = [
    (0.1, 0.001, 0.01),
    (0.1, 0.05, 0.5),
    (0.2, 0.01, 0.1),
    (0.2, 0.08, 0.8),
    (0.5, 0.001, 0.01)
]
for tau, l1, l2 in fimap_hyperparameters:
    fimap = Fimap(tau, l1, l2)
    fimap.fit(german_data.X_train, model_predictions, epochs=300)
    fimaps.append(fimap)
    
cadexs = []
n_list = [5, 8, 10, 15, 20]
for n_changed in n_list:
    cadex = Cadex(model, n_changed, transform=german_data.scale, inverse_transform=german_data.unscale)
    cadexs.append(cadex)

Now, let's use ECE and generate up to 4 best counterfactuals:

In [ ]:
from counterfactuals.explainers import ECE
from counterfactuals.visualization import compare
pd.set_option("display.max_rows", None)

ece = ECE(4, columns=list(german_data.X_train.columns), bces=cadexs + fimaps, dist=2, h=5, lambda_=0.001, n_jobs=1)
cfs = ece.generate(X_test)

First, let's see all 10 counterfactuals:

In [22]:
compare(X_test, ece.get_aggregated_cfs())

,X,CF1 change,CF2 change,CF3 change,CF4 change,CF5 change,CF6 change,CF7 change,CF8 change,CF9 change,CF10 change,constraint
number of attributes changed,0.0,61.000,61.000,61.000,61.000,61.000,20.0,15.0,10.0,8.0,5.0,
duration,6.0,-41.557,-39.434,-39.420,-34.842,-32.236,8.671,10.694,14.816,17.161,24.392,
credit,1169.0,-73.950,-30.255,-41.122,-26.840,-24.256,2000.807,2467.624,3418.899,-,-,
installment_percent,4.0,-30.909,-32.632,-26.025,-10.373,-15.488,-,-,-,-,-,
residence_duration,4.0,39.600,36.821,33.918,11.724,19.915,-,-,-,-,-,
age,67.0,57.382,42.105,41.579,21.242,33.334,-8.268,-10.197,-14.127,-16.363,-,
existing_credits,2.0,47.064,28.415,-18.560,-1.171,9.891,-,-,-,-,-,
people_maintained,1.0,-17.572,-24.881,-31.594,-5.063,-7.778,-,-,-,-,-,
account_status_0..200 DM,0.0,-43.428,-42.611,-45.683,-23.750,-23.730,0.328,0.404,-,-,-,
account_status_< 0 DM,1.0,-49.541,-41.874,-39.607,-30.512,-25.856,0.329,0.406,0.562,0.651,0.925,


And now, those selected by ECE:

In [23]:
compare(X_test, cfs)

,X,CF1 change,CF2 change,constraint
number of attributes changed,0.0,15.0,10.0,
duration,6.0,10.694,14.816,
credit,1169.0,2467.624,3418.899,
installment_percent,4.0,-,-,
residence_duration,4.0,-,-,
age,67.0,-10.197,-14.127,
existing_credits,2.0,-,-,
people_maintained,1.0,-,-,
account_status_0..200 DM,0.0,0.404,-,
account_status_< 0 DM,1.0,0.406,0.562,


We can do the same with constraints:

In [ ]:
fimaps = []
fimap_hyperparameters = [
    (0.1, 0.001, 0.01),
    (0.1, 0.05, 0.5),
    (0.2, 0.01, 0.1),
    (0.2, 0.08, 0.8),
    (0.5, 0.001, 0.01)
]
for tau, l1, l2 in fimap_hyperparameters:
    fimap = Fimap(tau, l1, l2, constraints=constraints)
    fimap.fit(german_data.X_train, model_predictions, epochs=300)
    fimaps.append(fimap)
    
cadexs = []
n_list = [10, 14, 18, 20, 25]
for n_changed in n_list:
    cadex = Cadex(model, n_changed, transform=german_data.scale, inverse_transform=german_data.unscale, constraints=constraints)
    cadexs.append(cadex)

In [25]:
ece = ECE(4, columns=list(german_data.X_train.columns), bces=cadexs + fimaps, dist=2, h=5, lambda_=0.001, n_jobs=1)
cfs = ece.generate(X_test)

In [26]:
compare(X_test, ece.get_aggregated_cfs(), constraints=constraints)

,X,CF1 change,CF2 change,CF3 change,CF4 change,CF5 change,CF6 change,CF7 change,CF8 change,CF9 change,CF10 change,constraint
number of attributes changed,0.0,17.0,21.0,19.0,16.0,19.0,14.0,14.0,13.0,8.0,6.0,
duration,6.0,-53.748,-42.422,-41.875,-37.604,-31.255,13.671,13.671,13.671,13.838,15.716,
credit,1169.0,-57.972,-30.599,-35.715,-27.36,-20.739,3154.589,3154.596,3154.615,3193.211,3626.684,
installment_percent,4.0,-42.054,-11.962,-28.802,-20.783,-9.934,1.289,1.289,1.289,-,-,
residence_duration,4.0,26.374,17.231,23.408,25.461,16.667,-1.273,-1.273,-1.273,-,-,
age,67.0,20.537,24.236,25.516,-,4.315,-,-,-,-,-,Monotonicity increasing
existing_credits,2.0,41.686,0.312,-0.124,1.187,12.889,0.679,0.679,-,-,-,
people_maintained,1.0,-39.332,-8.426,-14.234,-20.066,-8.382,0.421,0.421,0.421,-,-,
account_status_0..200 DM,0.0,-,-,-,1.0,-,-,-,-,-,-,OneHot
account_status_< 0 DM,1.0,-1.0,-1.0,-1.0,-1.0,-,-,-,-,-,-,OneHot


In [27]:
compare(X_test, cfs, constraints=constraints)

,X,CF1 change,CF2 change,constraint
number of attributes changed,0.0,14.0,8.0,
duration,6.0,13.671,13.838,
credit,1169.0,3154.596,3193.211,
installment_percent,4.0,1.289,-,
residence_duration,4.0,-1.273,-,
age,67.0,-,-,Monotonicity increasing
existing_credits,2.0,0.679,-,
people_maintained,1.0,0.421,-,
account_status_0..200 DM,0.0,-,-,OneHot
account_status_< 0 DM,1.0,-,-,OneHot


For more information on the library, see our documentation: https://counterfactuals.readthedocs.io/en/latest/